In [4]:
import torch
from transformers import pipeline
from optparse import OptionParser
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json

In [5]:
json_path="/home/bankerz/Downloads/01_test.json"

with open(json_path) as json_file:
    schema_dictionary = json.load(json_file)

In [12]:
def schemaToDictionary(schemaList):
    '''
        Takes nodes or edges list and makes a dictionary you can index list elements using the ID, probably
        only worth running this function if you have a really big graph you need to access nodes or edges of
        many times, but this option is available for that
    '''
    newDict={}
    for dictionary in schemaList:
        newDict[dictionary['id']] = dictionary
    return newDict

In [22]:
schema_dictionary['edges']
#im pretty sure this flow was not made properly, to do the desired effect correctly can circumvent problem by closing loop from bottom to
#top, but this might be very confusing for users who will expect source and target to not be dependent on which side but on
#which node they drew the edge from, our first version will be side dependent but future versions should figure out on 
#client side where the edge started from and is going to source and target edges properly
#there should also be some visual indication of where inputs and outputs are going, again just publish the website quickly and fix this
#in an update

[{'id': 'e1-2', 'source': '1', 'target': '2'},
 {'source': '00001',
  'sourceHandle': 'bottom',
  'target': '3159559',
  'targetHandle': 'top',
  'id': 'reactflow__edge-00001bottom-3159559top'},
 {'source': '00001',
  'sourceHandle': 'right',
  'target': '3166128',
  'targetHandle': 'left',
  'id': 'reactflow__edge-00001right-3166128left'},
 {'source': '3166128',
  'sourceHandle': 'bottom',
  'target': '3171323',
  'targetHandle': 'top',
  'id': 'reactflow__edge-3166128bottom-3171323top'},
 {'source': '3159559',
  'sourceHandle': 'right',
  'target': '3171323',
  'targetHandle': 'left',
  'id': 'reactflow__edge-3159559right-3171323left'}]

In [15]:
test = schemaToDictionary(schema_dictionary['nodes'])

In [20]:
test['00001']

{'id': '00001',
 'type': 'textInput',
 'data': {'label': 'You are Frodo. Say something interesting, and respond to text above if you received any.'},
 'position': {'x': 195.185336410498, 'y': 46.87644239300134},
 'width': 289,
 'height': 50,
 'selected': False,
 'positionAbsolute': {'x': 195.185336410498, 'y': 46.87644239300134},
 'dragging': False}

In [ ]:
#to start with, I think we should always start from the first node created. May make sense in the future to allow user to specify
#what node they want to start with, but i am too lazy to think of what the UX of that would be 

In [ ]:
def runSchema(schema_dictionary):
    '''
        Take a schema and run the flow. This version will simply start from the first node.
        The first node is special, it will always be run even if it has souces that have not
        been given a prompt yet, if it does have sources without a prompt, the code should
        place itself at the back of the stack after running and finding the next members of the
        stack.
        For this idea, we should create a stack, a list of dictionaries that look like this:
        [{node_id: ____, sources_dict:{source_id:received_prompt = "" }}]
        This forms a stack. Sources dict may or may not have multiple source_ids in it. Queue up
        the next nodes to run based on what the targets of the node you just ran are. If
        a node has multiple sources when it's run, combine those sources into one prompt
        via concatenation. Received prompts are added first, with the LLMs actual text
        entered into it added last. We may have to do prompt engineering to make sure
        the LLM answers the prompt inputted into the box and not questions received as context,
        but this is not preferred.
        For bonus points, add an option to use dfs or bfs
    '''